<a href="https://colab.research.google.com/github/dapopov-st/MiniLlamaSQL/blob/main/Fine_tune_CodeLlama_7b_Instruct_step_by_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune premsql

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


/home/mainuser/anaconda3/envs/sqlft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-10 14:46:58,779] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/mainuser/anaconda3/envs/sqlft/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
print(torch.cuda.is_available())

True


In [3]:
import wandb # do from scipt later
wandb.login()
%env WANDB_PROJECT=sql-fine-tuning

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dpopovvelasco. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=sql-fine-tuning


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
output_dir = './sql_output_dir_premsql'
logging_dir = './sql_logging_dir_premsql'

In [6]:
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
clean_mem()

## Load and Prepare the Data

<|begin▁of▁sentence|> Question: What is the total sales amount? Context: The sales table contains information about each sale including date, amount, and customer ID. Answer: SELECT SUM(amount) FROM sales; <|end▁of▁sentence|>

In [10]:

from transformers import AutoTokenizer
from datasets import load_dataset

# Load a tokenizer to use its chat template
template_tokenizer = AutoTokenizer.from_pretrained(
    "premai-io/prem-1B-SQL"
)

# def format_prompt(sample):

#     """Given a sample dictionary with keys "title" and "abstract" format into a prompt.

#     Args:
#       sample: A sample dictionary from a Hugging Face dataset.

#     Returns:
#       sample: sample dictionary with "text" key for the formatted prompt.
#     """
#     #sample['text']=f"[INST] <> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <> {sample['question']} {sample['context']} [/INST] {sample['answer']}"
#     sample['text']=f"""<|begin▁of▁sentence|> You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.
# Context: {sample['context']}
# Question: {sample['question']}
# SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
#     return {"text":sample['text']}

def format_prompt(sample):
#     sample['text']=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
# You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>
# Context: {sample['context']}
# Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
    sample['text'] = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {sample['context']}
Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
    return {"text":sample['text']}
  

# Load and format the data using the template TinyLLama is using
dataset = (
    load_dataset("dpv/exoplanets-sql")#, split="train")
      .shuffle(seed=42)
)
dataset = dataset.map(format_prompt)

Map: 100%|██████████| 10/10 [00:00<00:00, 3414.72 examples/s]


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 10
    })
})

In [12]:
dataset['train']['text'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: CREATE TABLE exoplanets (\n    name TEXT,\n    distance REAL,\n    stellar_magnitude REAL,\n    planet_type TEXT,\n    discovery_year INTEGER,\n    mass_multiplier REAL,\n    mass_wrt TEXT,\n    radius_multiplier REAL,\n    radius_wrt TEXT,\n    orbital_radius REAL,\n    orbital_period REAL,\n    eccentricity REAL,\n    detection_method TEXT\n); CREATE TABLE reference_planets (name TEXT, mass REAL);  'mass_wrt' in exoplanets table has a one-to-one match to 'name' in reference_planets table, and  'name' refers to either Earth or Jupyter, with Jupyter having 317.8 the mass of Earth.\nQuestion: What is the average mass of exoplanets discovered each year?<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:SELECT \n    e.discovery_y

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 10
    })
})

- SFTTrainer expect a single 'text' column

## Evaluate the original model

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODEL_ID="premai-io/prem-1B-SQL"
model_orig = AutoModelForCausalLM.from_pretrained(MODEL_ID).to("cuda:1")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.86it/s]


In [15]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM exoplanets LIMIT 5;")
def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)
def compare_results(query_results, expected_results):
    """Compare the query results with the expected results."""
    # Sort the results for comparison
    query_results_sorted = sorted(query_results, key=lambda x: json.dumps(x, sort_keys=True))
    expected_results_sorted = sorted(expected_results, key=lambda x: json.dumps(x, sort_keys=True))
    return query_results_sorted == expected_results_sorted

valid_loaded = load_dataset('json', data_files='./databases/valid.json', split='train')

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model_orig.eval()
    with torch.no_grad():
        out = tokenizer.decode(model_orig.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    res = out[out.find('SQL Query:')+10:]
    #print(out[out.find(';')])
    if debug: print(res)
    run_query(res, cursor)
    return res
num_opp_errors = 0
num_other_exceptions = 0
op_errors = []
exceptions  = []
generations = []
results = []
bad_idxs = []
result_comparisons = [False]*len(valid_loaded)
for idx,example in enumerate(valid_loaded):
    generation=''
    result = None
    try:
        print('idx: ',idx)
        generation=generate_sql(context=example['context'], question=example['question'],debug=True)
        result = run_query(generation,cursor)
    except sqlite3.OperationalError as op_err:
        num_opp_errors += 1
        op_errors.append((idx,op_err))
        bad_idxs.append(idx)
    except Exception as e:
        num_other_exceptions +=1
        exceptions.append((idx,e))
        bad_idxs.append(idx)
    else:
        result_comparisons[idx]=compare_results(result, example['results'])
    finally:
        results.append(result)
        generations.append(generation)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  0


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT discovery_year, COUNT(*) as num_exoplanets FROM exoplanets GROUP BY discovery_year ORDER BY num_exoplanets DESC LIMIT 1;

idx:  1


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

 SELECT name, mass, discovery_year FROM (SELECT name, mass, discovery_year, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn FROM exoplanets) t WHERE rn = 1;  'rn' is the row number assigned by the ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn. If rn = 1, then the mass of the exoplanet is the minimum for that discovery year.

idx:  7


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT name, discovery_year, detection_method FROM exoplanets WHERE discovery_year BETWEEN (SELECT MAX(discovery_year) - 2 FROM exoplanets) AND (SELECT MAX(discovery_year) FROM exoplanets);
idx:  8


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius FROM exoplanets GROUP BY planet_type ORDER BY avg_orbital_radius DESC;
idx:  9
 SELECT name, (mass_wrt::REAL / radius_wrt::REAL) AS ratio FROM exoplanets ORDER BY ratio DESC LIMIT 5;  'mass_wrt' and 'radius_wrt' are the mass and radius of the exoplanet in the reference_planets table, respectively.

num_opp_errors: 3, num_other_exceptions:2,
Error rate: 50.0


In [16]:
op_errors, exceptions, result_comparisons

([(2, sqlite3.OperationalError('no such column: mass')),
  (6, sqlite3.OperationalError('no such column: mass')),
  (9, sqlite3.OperationalError('unrecognized token: ":"'))],
 [(3,
   sqlite3.ProgrammingError('You can only execute one statement at a time.')),
  (5,
   sqlite3.ProgrammingError('You can only execute one statement at a time.'))],
 [False, False, False, False, False, False, False, False, True, False])

In [17]:
generations

[' SELECT discovery_year, COUNT(*) as num_exoplanets FROM exoplanets GROUP BY discovery_year ORDER BY num_exoplanets DESC LIMIT 1;\n',
 ' SELECT detection_method FROM exoplanets WHERE mass_wrt = (SELECT name FROM reference_planets WHERE mass = 317.8) GROUP BY detection_method ORDER BY COUNT(*) DESC LIMIT 1;\n',
 '',
 '',
 ' SELECT * FROM exoplanets WHERE discovery_year < 2000 AND eccentricity < 0.2;',
 '',
 '',
 ' SELECT name, discovery_year, detection_method FROM exoplanets WHERE discovery_year BETWEEN (SELECT MAX(discovery_year) - 2 FROM exoplanets) AND (SELECT MAX(discovery_year) FROM exoplanets);',
 ' SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius FROM exoplanets GROUP BY planet_type ORDER BY avg_orbital_radius DESC;',
 '']

In [19]:
bad_idxs

[2, 3, 5, 6, 9]

In [21]:
# for idx in bad_idxs:
#     print(run_query(generations[idx],cursor))
#     print(valid_loaded[idx]['results'])
#     print('_'*100)
# #generations[2]

In [38]:
#run_query("""SELECT name, mass, discovery_year FROM (SELECT name, mass, discovery_year, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn FROM exoplanets) t WHERE rn = 1;""",cursor)
run_query("""SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY orbital_period DESC) as rn FROM exoplanets) t WHERE rn <= 3; """,cursor)


'[{"name": "PSR B1257+12 d", "distance": 1957.0, "stellar_magnitude": null, "planet_type": "Super Earth", "discovery_year": 1992, "mass_multiplier": 3.9, "mass_wrt": "Earth", "radius_multiplier": 1.8, "radius_wrt": "Earth", "orbital_radius": 0.46, "orbital_period": 0.26885694, "eccentricity": 0.03, "detection_method": "Pulsar Timing", "rn": 1}, {"name": "PSR B1257+12 c", "distance": 1957.0, "stellar_magnitude": null, "planet_type": "Super Earth", "discovery_year": 1992, "mass_multiplier": 4.3, "mass_wrt": "Earth", "radius_multiplier": 1.91, "radius_wrt": "Earth", "orbital_radius": 0.36, "orbital_period": 0.18206708, "eccentricity": 0.02, "detection_method": "Pulsar Timing", "rn": 2}, {"name": "PSR B1257+12 b", "distance": 1957.0, "stellar_magnitude": null, "planet_type": "Terrestrial", "discovery_year": 1994, "mass_multiplier": 0.02, "mass_wrt": "Earth", "radius_multiplier": 0.338, "radius_wrt": "Earth", "orbital_radius": 0.19, "orbital_period": 0.06926762, "eccentricity": 0.0, "detect

## Configure Model, Data, Tokenizer, Trainer

In [22]:
clean_mem()

In [23]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_config,get_peft_model
MODEL_ID="premai-io/prem-1B-SQL" # just released Sept. 25, 2024, waiting for access!

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map='auto',
    #quantization_config = bnb_config
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = "<pad>"
tokenizer.padding_side = "left"
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=128,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # Layers to target
     ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, peft_config)


Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.56it/s]


In [24]:
from transformers import DataCollatorForLanguageModeling
# Step 3: Launch training with SFTTrainer
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs = 10.0,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
   # peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

trainer.train()


Generating train split: 86 examples [00:00, 555.80 examples/s]
Generating train split: 16 examples [00:00, 5580.78 examples/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


 10%|█         | 10/100 [00:09<01:18,  1.14it/s]

{'loss': 0.8543, 'grad_norm': 1.726805567741394, 'learning_rate': 0.00019510565162951537, 'epoch': 0.93}


 20%|██        | 20/100 [00:17<01:09,  1.16it/s]

{'loss': 0.262, 'grad_norm': 1.6280531883239746, 'learning_rate': 0.00018090169943749476, 'epoch': 1.86}


...

100%|██████████| 100/100 [01:27<00:00,  1.15it/s]

{'loss': 0.0125, 'grad_norm': 0.3487114906311035, 'learning_rate': 4.934396342684e-08, 'epoch': 9.3}


100%|██████████| 100/100 [01:37<00:00,  1.03it/s]

{'train_runtime': 97.7084, 'train_samples_per_second': 8.802, 'train_steps_per_second': 1.023, 'train_loss': 0.16610419526696205, 'epoch': 9.3}


TrainOutput(global_step=100, training_loss=0.16610419526696205, metrics={'train_runtime': 97.7084, 'train_samples_per_second': 8.802, 'train_steps_per_second': 1.023, 'total_flos': 1573369833062400.0, 'train_loss': 0.16610419526696205, 'epoch': 9.30232558139535})

In [27]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)
def compare_results(query_results, expected_results):
    """Compare the query results with the expected results."""
    # Sort the results for comparison
    query_results_sorted = sorted(query_results, key=lambda x: json.dumps(x, sort_keys=True))
    expected_results_sorted = sorted(expected_results, key=lambda x: json.dumps(x, sort_keys=True))
    return query_results_sorted == expected_results_sorted

valid_loaded = load_dataset('json', data_files='./databases/valid.json', split='train')

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model.eval()
    with torch.no_grad():
        out = tokenizer.decode(model.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    #res = out[out.find('SQL Query:')+10:]
    res = out[out.find('SQL Query:')+10:out.find('<|end▁of▁sentence|>')]

    #print(out[out.find(';')])
    if debug: print(res)
    run_query(res, cursor)
    return res
num_opp_errors = 0
num_other_exceptions = 0
op_errors = []
exceptions  = []
generations = []
results = []
bad_idxs = []
result_comparisons = [False]*len(valid_loaded)
for idx,example in enumerate(valid_loaded):
    generation=''
    result = None
    try:
        print('idx: ',idx)
        generation=generate_sql(context=example['context'], question=example['question'],debug=True)
        result = run_query(generation,cursor)
    except sqlite3.OperationalError as op_err:
        num_opp_errors += 1
        op_errors.append((idx,op_err))
        bad_idxs.append(idx)
    except Exception as e:
        num_other_exceptions +=1
        exceptions.append((idx,e))
        bad_idxs.append(idx)
    else:
        result_comparisons[idx]=compare_results(result, example['results'])
    finally:
        results.append(result)
        generations.append(generation)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  0


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


WITH YearlyCounts AS (
    SELECT 
        discovery_year,
        COUNT(*) AS num_exoplanets,
        MIN(discovery_year) AS min_discovery_year
    FROM exoplanets
    GROUP BY discovery_year
)
SELECT 
    discovery_year,
    num_exoplanets,
    min_discovery_year
FROM YearlyCounts
ORDER BY num_exoplanets DESC
LIMIT 1; 
idx:  1


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

WITH MinContexts AS (
    SELECT 
        discovery_year,
        COUNT(*) AS num_exoplanets,
        MIN(distance) AS min_distance
    FROM exoplanets
    GROUP BY discovery_year
)
SELECT 
    e.name,
    e.distance
FROM exoplanets e
JOIN MinContexts m ON e.discovery_year = m.discovery_year; 
idx:  7
SELECT e.name, e.distance, e.stellar_magnitude, e.discovery_year, e.detection_method
FROM exoplanets e
WHERE e.distance IS NOT NULL and e.stellar_magnitude IS NOT NULL
ORDER BY e.distance ASC, e.stellar_magnitude ASC;  


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  8


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius
FROM exoplanets
GROUP BY planet_type
ORDER BY avg_orbital_radius DESC; 
idx:  9
WITH RankedExoplanets AS (
    SELECT 
        e.name,
        e.mass_multiplier * r.mass AS mass,
        e.discovery_year,
        RANK() OVER (PARTITION BY e.discovery_year ORDER BY e.mass_multiplier * r.mass DESC) AS mass_rank
    FROM exoplanets e
    JOIN reference_planets r ON e.mass_wrt = r.name
)
SELECT 
    e.name,
    e.mass_multiplier * r.mass AS mass,
    e.discovery_year
FROM RankedExoplanets
WHERE mass_rank <= 5; 
num_opp_errors: 3, num_other_exceptions:0,
Error rate: 30.0


In [28]:
op_errors, exceptions,result_comparisons

([(3, sqlite3.OperationalError('no such column: e.name')),
  (5, sqlite3.OperationalError('no such column: planet_type')),
  (9, sqlite3.OperationalError('no such column: e.name'))],
 [],
 [False, False, False, False, False, False, False, False, True, False])

- Not bad, brought error rate down from 50% to 20-30% correct query formatting within 10 epochs/2 minutes of training.  Results don't yield the same answers, though!  Could be lack of knowledge about the prompt structure for premsql, or perhaps the questions in the validation set need to be made clearer.

In [29]:
valid_loaded['answer']

['SELECT discovery_year, COUNT(*) AS count\nFROM exoplanets\nGROUP BY discovery_year\nORDER BY count DESC\nLIMIT 1;',
 'SELECT detection_method,\n       SUM(mass_multiplier * (SELECT mass FROM reference_planets WHERE name = mass_wrt)) AS total_mass\nFROM exoplanets\nGROUP BY detection_method\nORDER BY total_mass DESC\nLIMIT 1;',
 'SELECT discovery_year, \nSUM(mass_multiplier * (SELECT mass FROM reference_planets WHERE name = mass_wrt)) AS total_mass\nFROM exoplanets\nWHERE mass_multiplier IS NOT NULL\nGROUP BY discovery_year\nORDER BY total_mass DESC\nLIMIT 5;',
 'WITH RankedExoplanets AS ( SELECT discovery_year, name, orbital_period, RANK() OVER (PARTITION BY discovery_year ORDER BY orbital_period DESC) AS period_rank FROM exoplanets) SELECT * FROM RankedExoplanets WHERE period_rank <= 3;',
 'SELECT name, discovery_year, eccentricity\nFROM exoplanets\nWHERE discovery_year < 2000 AND eccentricity < 0.2;',
 'SELECT DISTINCT \n    planet_type\nFROM \n    exoplanets\nWHERE \n    planet_ty

In [30]:
generations

['WITH YearlyCounts AS (\n    SELECT \n        discovery_year,\n        COUNT(*) AS num_exoplanets,\n        MIN(discovery_year) AS min_discovery_year\n    FROM exoplanets\n    GROUP BY discovery_year\n)\nSELECT \n    discovery_year,\n    num_exoplanets,\n    min_discovery_year\nFROM YearlyCounts\nORDER BY num_exoplanets DESC\nLIMIT 1; ',
 'WITH MethodCounts AS (\n    SELECT \n        detection_method,\n        COUNT(*) AS method_count\n    FROM exoplanets\n    GROUP BY detection_method\n)\nSELECT \n    detection_method,\n    method_count\nFROM MethodCounts\nORDER BY method_count DESC\nLIMIT 1; ',
 'WITH YearlyMass AS (\n    SELECT \n        discovery_year,\n        COUNT(*) AS total_mass\n    FROM exoplanets\n    GROUP BY discovery_year\n)\nSELECT \n    discovery_year,\n    total_mass\nFROM YearlyMass\nORDER BY total_mass DESC\nLIMIT 5; ',
 '',
 'SELECT \n    e.name,\n    e.discovery_year,\n    e.eccentricity,\n    e.mass_multiplier * r.mass AS mass\nFROM exoplanets e\nJOIN reference_

- Results for query 0 are equivalent, just extra column in generated query (and it's a bit too complex):

In [40]:
i=0
valid_result=valid_loaded['results'][0]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[0],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"discovery_year": 2016, "count": 1517}]
gen_result:
 [{"discovery_year": 2016, "num_exoplanets": 1517, "min_discovery_year": 2016}]
How are the following two queries different? WITH YearlyCounts AS (
    SELECT 
        discovery_year,
        COUNT(*) AS num_exoplanets,
        MIN(discovery_year) AS min_discovery_year
    FROM exoplanets
    GROUP BY discovery_year
)
SELECT 
    discovery_year,
    num_exoplanets,
    min_discovery_year
FROM YearlyCounts
ORDER BY num_exoplanets DESC
LIMIT 1; , SELECT discovery_year, COUNT(*) AS count
FROM exoplanets
GROUP BY discovery_year
ORDER BY count DESC
LIMIT 1; and which one better answers the following question: Determine the number of exoplanets discovered each year, and show the year with the highest number of discoveries.?


- Query 1 is off track:

In [41]:
i=1
valid_result=valid_loaded['results'][i]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[i],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"detection_method": "Radial Velocity", "total_mass": 6.38296985206948e+30}]
gen_result:
 [{"detection_method": "Transit", "method_count": 3945}]
How are the following two queries different? WITH MethodCounts AS (
    SELECT 
        detection_method,
        COUNT(*) AS method_count
    FROM exoplanets
    GROUP BY detection_method
)
SELECT 
    detection_method,
    method_count
FROM MethodCounts
ORDER BY method_count DESC
LIMIT 1; , SELECT detection_method,
       SUM(mass_multiplier * (SELECT mass FROM reference_planets WHERE name = mass_wrt)) AS total_mass
FROM exoplanets
GROUP BY detection_method
ORDER BY total_mass DESC
LIMIT 1; and which one better answers the following question: Display the detection method that has produced the most massive exoplanets (based on total mass).?


- Query 2 is close, but off track:

In [42]:
i=2
valid_result=valid_loaded['results'][i]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[i],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"discovery_year": 2016, "total_mass": 2.3398372025261199e+30}, {"discovery_year": 2014, "total_mass": 1.92648023608052e+30}, {"discovery_year": 2022, "total_mass": 1.8723379260193998e+30}, {"discovery_year": 2021, "total_mass": 8.9332245596716e+29}, {"discovery_year": 2011, "total_mass": 6.552292609403601e+29}]
gen_result:
 [{"discovery_year": 2016, "total_mass": 1517}, {"discovery_year": 2014, "total_mass": 875}, {"discovery_year": 2021, "total_mass": 525}, {"discovery_year": 2022, "total_mass": 338}, {"discovery_year": 2018, "total_mass": 326}]
How are the following two queries different? WITH YearlyMass AS (
    SELECT 
        discovery_year,
        COUNT(*) AS total_mass
    FROM exoplanets
    GROUP BY discovery_year
)
SELECT 
    discovery_year,
    total_mass
FROM YearlyMass
ORDER BY total_mass DESC
LIMIT 5; , SELECT discovery_year, 
SUM(mass_multiplier * (SELECT mass FROM reference_planets WHERE name = mass_wrt)) AS total_mass
FROM exoplanets
WHERE mass_multi

- Query 4 just gives extra information, can count it, though:

In [44]:
i=4
valid_result=valid_loaded['results'][i]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[i],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"name": "47 Ursae Majoris b", "discovery_year": 1996, "eccentricity": 0.03}, {"name": "51 Pegasi b", "discovery_year": 1995, "eccentricity": 0.01}, {"name": "55 Cancri b", "discovery_year": 1996, "eccentricity": 0.0}, {"name": "GJ 86 b", "discovery_year": 1999, "eccentricity": 0.04}, {"name": "GJ 876 b", "discovery_year": 1998, "eccentricity": 0.03}, {"name": "HD 10697 b", "discovery_year": 1999, "eccentricity": 0.1}, {"name": "HD 130322 b", "discovery_year": 1999, "eccentricity": 0.03}, {"name": "HD 177830 b", "discovery_year": 1999, "eccentricity": 0.1}, {"name": "HD 187123 b", "discovery_year": 1998, "eccentricity": 0.01}, {"name": "HD 192263 b", "discovery_year": 1999, "eccentricity": 0.05}, {"name": "HD 195019 b", "discovery_year": 1998, "eccentricity": 0.01}, {"name": "HD 209458 b", "discovery_year": 1999, "eccentricity": 0.0}, {"name": "HD 217107 b", "discovery_year": 1998, "eccentricity": 0.13}, {"name": "HD 75289 b", "discovery_year": 1999, "eccentricity": 0.0

- Query 6 goes off track, focusing on min distance instead of mass:

In [46]:
i=6
valid_result=valid_loaded['results'][i]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[i],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"discovery_year": 2001, "name": "47 Ursae Majoris c", "exoplanet_mass": 1.02492e+27}, {"discovery_year": 1995, "name": "51 Pegasi b", "exoplanet_mass": 8.730800000000001e+26}, {"discovery_year": 2004, "name": "55 Cancri e", "exoplanet_mass": 4.771628000000001e+25}, {"discovery_year": 2019, "name": "EPIC 201497682 b", "exoplanet_mass": 1.5527200000000002e+24}, {"discovery_year": 2007, "name": "GJ 581 c", "exoplanet_mass": 3.2846e+25}, {"discovery_year": 2009, "name": "GJ 581 e", "exoplanet_mass": 1.01524e+25}, {"discovery_year": 1998, "name": "HD 187123 b", "exoplanet_mass": 9.926540000000001e+26}, {"discovery_year": 2003, "name": "HD 3651 b", "exoplanet_mass": 4.32744e+26}, {"discovery_year": 2000, "name": "HD 46375 b", "exoplanet_mass": 4.28948e+26}, {"discovery_year": 2002, "name": "HD 49674 b", "exoplanet_mass": 1.89808076e+26}, {"discovery_year": 2006, "name": "HD 69830 b", "exoplanet_mass": 6.09144e+25}, {"discovery_year": 1999, "name": "HD 75289 b", "exoplanet_ma

- Query 7 does not focus on recent discoveries:

In [47]:
i=7
valid_result=valid_loaded['results'][i]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[i],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"name": "GJ 1151 c", "discovery_year": 2023, "detection_method": "Radial Velocity"}, {"name": "TOI-1669 b", "discovery_year": 2023, "detection_method": "Radial Velocity"}, {"name": "TOI-1694 b", "discovery_year": 2023, "detection_method": "Transit"}, {"name": "TOI-1694 c", "discovery_year": 2023, "detection_method": "Radial Velocity"}, {"name": "TOI-4342 b", "discovery_year": 2023, "detection_method": "Transit"}, {"name": "TOI-4342 c", "discovery_year": 2023, "detection_method": "Transit"}, {"name": "TOI-4562 b", "discovery_year": 2023, "detection_method": "Transit"}, {"name": "TOI-4582 b", "discovery_year": 2023, "detection_method": "Transit"}, {"name": "TOI-700 e", "discovery_year": 2023, "detection_method": "Transit"}, {"name": "AB Aurigae b", "discovery_year": 2022, "detection_method": "Direct Imaging"}, {"name": "CoRoT-35 b", "discovery_year": 2022, "detection_method": "Transit"}, {"name": "CoRoT-36 b", "discovery_year": 2022, "detection_method": "Transit"}, {"nam

- Query 8 is good:

In [48]:
i=8
valid_result=valid_loaded['results'][i]
print('valid_result:\n',valid_result)
gen_result=run_query(generations[i],cursor)
print('gen_result:\n', gen_result)
compare_results(valid_result,gen_result)
print(f"""How are the following two queries different? {generations[i]}, {valid_loaded['answer'][i]} and which one better answers the following question: {valid_loaded['question'][i]}?""")

valid_result:
 [{"planet_type": "Gas Giant", "avg_orbital_radius": 21.5154491121673}, {"planet_type": "Unknown", "avg_orbital_radius": 16.65}, {"planet_type": "Neptune-like", "avg_orbital_radius": 0.2249016860670194}, {"planet_type": "Super Earth", "avg_orbital_radius": 0.10995212107023411}, {"planet_type": "Terrestrial", "avg_orbital_radius": 0.06238086486486486}]
gen_result:
 [{"planet_type": "Gas Giant", "avg_orbital_radius": 21.5154491121673}, {"planet_type": "Unknown", "avg_orbital_radius": 16.65}, {"planet_type": "Neptune-like", "avg_orbital_radius": 0.2249016860670194}, {"planet_type": "Super Earth", "avg_orbital_radius": 0.10995212107023411}, {"planet_type": "Terrestrial", "avg_orbital_radius": 0.06238086486486486}]
How are the following two queries different? SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius
FROM exoplanets
GROUP BY planet_type
ORDER BY avg_orbital_radius DESC; , SELECT 
    e.planet_type,
    AVG(e.orbital_radius) AS avg_orbital_radius
FROM exopla

- Post fine tuning, can get 70% execution rate (up from 50%) and ~30% accuracy (up from 10%) with premsql.  The big unknown is exact the prompt format used by premsql, which I could not exactly determine.  Perhaps fixing this could lead to much better results.